In [1]:
install.packages('googledrive')
library(googledrive)

In [ ]:
install.packages("R.utils")
library("R.utils")
library("httr")
my_check <- function() {return(TRUE)}
reassignInPackage("is_interactive", pkgName = "httr", my_check)
options(rlang_interactive=TRUE)

In [ ]:
drive_auth(use_oob = TRUE, cache = TRUE)

In [ ]:
x <- drive_get("~/imdb/title.basics.tsv")
drive_download(x)
x <- drive_get("~/imdb/title.crew.tsv")
drive_download(x)
x <- drive_get("~/imdb/title.akas.tsv")
drive_download(x)
x <- drive_get("~/imdb/title.ratings.tsv")
drive_download(x)

In [ ]:
x <- drive_get("~/imdb/title.episode.tsv")
drive_download(x)
x <- drive_get("~/imdb/title.principals.tsv")
drive_download(x)
x <- drive_get("~/imdb/name_basics(1).tsv")
drive_download(x)

In [ ]:
install.packages('gridExtra')

In [ ]:
library('data.table')
library('dplyr')
library('tidyr')
library('stringr')
library('ggplot2')
library(gridExtra)
library('scales')

In [ ]:
data_basics <- fread("/content/title.basics.tsv")

In [ ]:
data_aka <- fread("/content/title.akas.tsv", quote = "")

In [ ]:
data_crew <- fread("/content/title.crew.tsv")

In [ ]:
data_ratings <- fread("/content/title.ratings.tsv")

In [ ]:
data_principals <- fread("/content/title.principals.tsv")

In [ ]:
data_episodes <- fread("/content/title.episode.tsv")

In [ ]:
data_names <-fread("/content/name_basics(1).tsv")

Merging

In [ ]:
tv_only <- data_basics %>% filter(titleType %in% c('tvSeries', 'tvEpisode', 'tvMiniSeries', 'tvPilot'))

In [ ]:
head(data_episodes)

In [ ]:
joined <- merge(tv_only, data_ratings, by.x = "tconst", 
             by.y = "tconst", all.x = FALSE, all.y = FALSE)

In [ ]:
count(names_filtered)

In [ ]:
head(data_names)

In [ ]:
names_filtered <- data_names %>% filter(nconst %in% data_principals$nconst)

aka_filtered <- data_aka %>% filter(titleId %in% joined$tconst)

crew_filtered <- data_crew %>% filter(tconst %in% joined$tconst)

data_principals <- data_principals %>% filter(tconst %in% joined$tconst)

In [ ]:
data_episodes <- data_episodes %>% filter(tconst %in% joined$tconst)

In [ ]:
head(data_principals)

In [ ]:
fwrite(crew_filtered, file = "/content/crew_filtered", sep="\t", na = "\\N", quote = FALSE, sep2 = c("",",",""))

In [ ]:
fwrite(joined, file = "/content/title_ratings", sep="\t", na = "\\N", quote = FALSE, sep2 = c("",",",""))

In [ ]:
fwrite(aka_filtered, file = "/content/aka_filtered", sep="\t", na = "\\N", quote = FALSE, sep2 = c("",",",""))

In [ ]:
fwrite(data_principals, file = "/content/principals_filtered", sep="\t", na = "\\N", quote = FALSE, sep2 = c("",",",""))

In [ ]:
fwrite(data_episodes, file = "/content/episodes_filtered", sep="\t", na = "\\N", quote = FALSE, sep2 = c("",",",""))

In [ ]:
fwrite(names_filtered, file = "/content/names_filtered", sep="\t", na = "\\N", quote = FALSE, sep2 = c("",",",""))

In [ ]:
gc()

# Title basics table
* tconst - primary ID of the title
* titleType - type of the title, char need to convert to category
* primaryTitle - primary title
* originalTitle - original title is different in ~ 135k cases, usually means translation from original language
* isAdult - need to convert to bool
* startYear - more than one milion movies without a start year
* endYear - only a 100k rows have it
* genres need to be converted to category

Convert \N char values to NA

In [ ]:
head(data_ratings, -30)

In [ ]:
data_basics <- data_basics %>% mutate(across(where(is.character), ~na_if(.,"\\N")))

In [ ]:
str(data_basics)

Number of NA's in each column

In [ ]:
sapply(data_basics, function(y) sum(length(which(is.na(y)))))

Only ~96k movies have end year

In [ ]:
sum(length(which(!is.na(data_basics$endYear))))

### Column type conversions
* convert title type to categorical
* converte genres to list of strings
* convert isAdult to Bool
* convert startYear, endYear, runtimeMinutes to int



In [ ]:
data_basics$titleType = as.factor(data_basics$titleType)
data_basics <- data_basics %>% mutate(genres = strsplit(genres, split=','))
data_basics$isAdult = as.logical(data_basics$isAdult)
data_basics$startYear = as.integer(data_basics$startYear)
data_basics$endYear = as.integer(data_basics$endYear)
data_basics$runtimeMinutes = as.integer(data_basics$runtimeMinutes)

**categories** - list of genres

In [ ]:
genres <- (data_basics %>% select(genres) %>% distinct())$genres %>% unlist() %>% unique()

In [ ]:
data_basics[sample(nrow(data_basics), 20), ]

**Title type** has no missing values or anomalies.

In [ ]:
title_types <- data_basics %>% select(titleType) %>% distinct()
title_types

Summary of the numerical columns:

In [ ]:
data_basics %>% select(startYear, endYear, runtimeMinutes) %>% summary()

Runtime minutes max looks sketchy, but we can see that it is just a weird movie.

In [ ]:
data_basics %>% arrange(desc(runtimeMinutes)) %>% head(10)

### Filtering


The titles without name are episode of a tv series so keeping them.

In [ ]:
data_basics %>% filter(is.na(primaryTitle))

In [ ]:
tmp <- data_basics %>% select(startYear, endYear) %>% filter(!(is.na(endYear) | is.na(startYear)))
year_diff <- tmp$endYear - tmp$startYear

We will use only one year, in the case that the movie has both start and end year we use the end year.

In [ ]:
data_basics <- data_basics %>% mutate(year = ifelse(is.na(endYear), startYear, endYear)) %>% select(-c(startYear, endYear))

##World events affecting movie production

---



In [ ]:
options(repr.plot.width=6, repr.plot.height=3, repr.plot.res = 250)

In [ ]:
comedies_30s <- data_basics %>% filter(year >= 1924 & year <= 1937) %>% mutate(new_col = lapply(genres, function (x) 'Comedy' %in% x)) %>% filter(new_col == TRUE)

In [ ]:
only_comedies_30s <- comedies_30s %>% mutate(new_col = lapply(genres, function (x) length(x) == 1)) %>% filter(new_col == TRUE)

In [ ]:
comedies_30s[sample(nrow(comedies_30s), 20)]

###The total movies produced during great depression vs total comedies

In [ ]:
comedies_30_freq <- as.data.frame(table(comedies_30s$year))

In [ ]:
movies_30_freq <- as.data.frame(table((data_basics %>% filter(year >= 1924 & year <= 1937))$year))

In [ ]:
comedies_30_freq
movies_30_freq

In [ ]:
share_comedies <- merge(x = comedies_30_freq, y = movies_30_freq, by="Var1") %>% mutate(share = ceiling(Freq.x / (Freq.y /100)))

In [ ]:
share_comedies$Var1 = as.numeric(as.character(share_comedies$Var1))

In [ ]:
plot_comedy_share <- ggplot(share_comedies, aes(x = Var1, y = share)) + 
geom_line(color='#eb3477', aes(group = 1)) + 
geom_point(color='black') +
scale_y_continuous(labels = function(x) paste0(x, "%")) +
geom_text(aes(1929.05, 27.3, label = 'Great depression begins'), size=3, color = '#505661', angle = 90) +
labs(y = 'Share of comedies', x = 'Year')

In [ ]:
plot_comedy_only <- (only_comedies_30s %>% select(year) %>% filter(year >= 1924 & year <= 1937) %>% ggplot(aes(x = year)) + 
geom_line(stat = 'count', color='blue') + 
geom_point(stat = 'count', color='black') +
geom_text(aes(1931.9, 95, label = 'Great depression begins'), size=3, color = '#505661') +
labs(y = 'Sole comedies', x = 'Year'))

In [ ]:
plot_comedy <- (comedies_30s %>% select(year) %>% filter(year >= 1924 & year <= 1937) %>% ggplot(aes(x = year)) + 
geom_line(stat = 'count', color='blue') + 
geom_point(stat = 'count', color='black') +
geom_text(aes(1930, 1050, label = 'Great depression begins'), size=3, color = '#505661') +
labs(y = 'Total comedies', x = 'Year'))

In [ ]:
plot_general <- (data_basics %>% select(year) %>% filter(year >= 1924 & year <= 1937) %>% ggplot(aes(x = year)) + 
geom_line(stat = 'count', color='orange') + 
geom_point(stat = 'count', color='black') +
geom_text(aes(1929, 3850, label = 'Great depression begins'), size=3, color = '#505661') +
labs(y = 'Total movies', x = 'Year'))

In [ ]:
options(repr.plot.width=8, repr.plot.height=4.2, repr.plot.res = 300)

In [ ]:
grid.arrange(plot_comedy, plot_general, plot_comedy_only, plot_comedy_share, ncol=2, top=textGrob("Movie and comedy production during great depression", x = 0.02, hjust = 0))

In [ ]:
data_basics %>% select(year) %>% filter(year >= 1935 & year <= 1948) %>% ggplot(aes(x = year)) + 
geom_line(stat = 'count', color='orange') + 
geom_point(stat = 'count', color='black') +
labs(y = 'Total movies produced', x = 'Year', title = 'Decline during WW2')

## Exploratory graphs

### Categories bar plot

In [ ]:
install.packages('viridis')

In [ ]:
library(viridis)

In [ ]:
genres = unlist(data_basics$genres)
genres_list = data.frame(genres)

In [ ]:
genres_list <- as.data.frame(table(genres_list))

In [ ]:
options(repr.plot.width=8, repr.plot.height=3, repr.plot.res = 300)

In [ ]:
ggplot(genres_list, aes(x=reorder(genres, -Freq), y = Freq, fill = Freq)) +
    geom_bar(stat = "identity") +
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
     scale_fill_viridis_c() +
     theme(legend.position="none") +
    scale_y_continuous(labels = unit_format(unit = "M", scale = 1e-6)) + xlab("")+
    ylab("Movies produced")

### Categories throughout years

In [ ]:
options(repr.plot.width=8, repr.plot.height=6, repr.plot.res = 300)

In [ ]:
chosen_genres <- c('Documentary','Animation','Comedy','Romance','Drama','Horror','War','Western','Family','Action','Sci-Fi','Talk-Show','Reality-TV','Adult')

In [ ]:
TT = theme(axis.text.x=element_blank(),
        axis.ticks.x=element_blank(),
        axis.title.x = element_blank(),
        axis.title.y = element_blank())
colors = c('firebrick4', 'blue4', 'yellowgreen', "orange", "gold4", "purple", "black",
 "chocolate1", "pink", "gray", "darksalmon", "coral4", "aquamarine4", "brown")
plotList <- function(df, name){
  tmp <- (ggplot(df, aes(x = year, y = Total)) +
  scale_y_continuous(labels = unit_format(unit = "K", scale = 1e-3))+
  geom_area(fill = colors[which(chosen_genres == name)])+
  geom_text(data=data.frame(), aes(label = name, x = -Inf, y = Inf), size = 6,
  color = colors[which(chosen_genres == name)], hjust = -0.3, vjust = 2))
  if (name %in% c('Reality-TV','Adult')) {
    tmp <- tmp + scale_x_continuous(breaks=seq(1910,2020,10)) +
            theme(axis.title.x = element_blank(),
                  axis.title.y = element_blank())
  } else {
    tmp <- tmp + TT
  }
  return(tmp)
}

In [ ]:
create_genre_plot <- function (genre) {
  temp <-
    ((data_basics %>% mutate(new_col = lapply(genres, function (x) genre %in% x)) %>% filter(new_col == TRUE) %>%
    filter(year >= 1910 & year <= 2022))$year %>% table() %>% as.data.frame %>% setNames(c("year", "Total")) %>%
    mutate(year = as.numeric(as.character(year))) %>% plotList(genre))
  return(temp)
}

In [ ]:
plots <- lapply(chosen_genres, create_genre_plot)

In [ ]:
grid.arrange(grobs = plots, ncol = 2, top = textGrob("Movie genre production (1910 - 2022)", x = 0.02, hjust = 0, gp=gpar(fontsize=20, col = 'gray25')))

### Title types pie chart

In [ ]:
title_df <- data_basics %>% 
  filter(!(titleType == 'tvEpisode')) %>%
  group_by(titleType) %>%
  count() %>% 
  ungroup() %>% 
  mutate(percentage = round(`n` / sum(`n`),3)) %>% 
  arrange(percentage) %>%
  mutate(labels = ifelse(percentage >= 0.06, scales::percent(percentage), ""))

In [ ]:
title_ep <- data_basics %>% mutate(new_col = ifelse(titleType == 'tvEpisode', 'TV episode', 'Other')) %>%
  group_by(new_col) %>%
  count() %>% 
  ungroup() %>% 
  mutate(percentage = round(`n` / sum(`n`),3) * 100)

In [ ]:
title_ep

In [ ]:
colnames(title_df)[1] <- "Title Type"

In [ ]:
title_df$percentage <- title_df$percentage * 100

In [ ]:
title_df <- title_df %>% arrange(-percentage)
title_df$`Title Type` <- factor(title_df$`Title Type`, levels=title_df$`Title Type`)

In [ ]:
title_df

In [ ]:
options(repr.plot.width=8, repr.plot.height=3, repr.plot.res = 300)

In [ ]:
pl1 <- ggplot(title_df[order(title_df$`Title Type`,decreasing=T),], aes(x = "", y = percentage, fill = `Title Type`)) +
  geom_col() +
  geom_text(aes(label = labels),
            position = position_stack(vjust = 0.5)) +
  scale_fill_brewer(palette = "Set3") +
  scale_y_continuous(labels = function(x) paste0(x, "%")) +
  theme_minimal(base_size = 16) +
    guides(fill=guide_legend(title="")) +
  ylab("") +
  xlab(NULL)

In [ ]:
pl2 <- ggplot(title_ep, aes(x = "", y = percentage, fill = new_col)) +
  geom_col() +
  geom_text(aes(label = paste0(percentage, "%")),
            position = position_stack(vjust = 0.5)) +
  scale_fill_brewer(palette = "Set3") +
  theme_minimal(base_size = 16) +
  guides(fill=guide_legend(title=""))+
  scale_y_continuous(labels = function(x) paste0(x, "%")) +
  ylab("") +
  xlab(NULL)

In [ ]:
grid.arrange(pl1, pl2, ncol=2)

In [ ]:
title_df

In [ ]:
gc()

# Title akas table

Replace \\N with NA

In [ ]:
data_aka <- data_aka %>% mutate(across(where(is.character), ~na_if(.,"\\N")))

**Transformations**
* types -> factor
* isOriginalTitle -> bool

In [ ]:
data_aka$types = as.factor(data_aka$types)
data_aka$isOriginalTitle <- as.logical(as.numeric(data_aka$isOriginalTitle))

**ordering** and **attributes** are not needed

In [ ]:
data_aka <- data_aka %>% select(-c(ordering, attributes))

In [ ]:
summary(data_aka %>% select(-c(titleId, title)))

In [ ]:
data_aka[sample(nrow(data_aka), 10), ]

In [ ]:
main_aka <- data_aka %>% filter(region %in% c('IT', 'US', 'GB', 'DE'))

### Adding country to the original titles

In [ ]:
originals <- data_aka %>% filter(isOriginalTitle)

In [ ]:
install.packages("cld2")
library(cld2)

In [ ]:
detect_language("Frozen Thrills are here")

In [ ]:
originals <- originals %>% mutate(country = detect_language(title))

In [ ]:
originals <- originals %>% filter(!is.na(country))

In [ ]:
originals <- originals %>% mutate(isEng = country == 'en')

In [ ]:
originals <- merge(originals, data_basics, by.x = 'titleId', by.y = 'tconst') %>% select (titleId, isEng, year)

In [ ]:
originals

In [ ]:
originals_temp <- originals %>% count(year)

In [ ]:
originals_perc <- merge(originals %>% count(year, isEng), originals_temp, by="year") %>%
filter(!is.na(year)) %>% mutate(percentages = n.x / (n.y / 100)) %>% filter(year >= 1910 & year <= 2020)

In [ ]:
ggplot(originals_perc, aes(x=year, y=percentages, fill=isEng)) +
    geom_area(alpha=0.6 , size=1, colour="black") +ylab("") + xlab("") + 
    scale_y_continuous(labels = function(x) paste0(x, "%")) +
    scale_fill_discrete(labels=c('English', 'Not english')) +
    guides(fill=guide_legend(title="Original language"))

### Merged akas and basics

In [ ]:
merged <- merge(data_basics, main_aka, by.x = 'tconst', by.y = 'titleId')

In [ ]:
merged <- merged %>% filter(year >= 1935 & year <= 1950)

In [ ]:
de <- as.data.frame(table((merged %>% mutate(new_col = lapply(genres, function (x) 'War' %in% x)) %>% filter(new_col == TRUE & region == 'DE'))$year)) %>% setNames(c('year', 'DE'))

In [ ]:
us <- as.data.frame(table((merged %>% mutate(new_col = lapply(genres, function (x) 'War' %in% x)) %>% filter(new_col == TRUE & region == 'US'))$year)) %>% setNames(c('year', 'US'))

In [ ]:
gb <- as.data.frame(table((merged %>% mutate(new_col = lapply(genres, function (x) 'War' %in% x)) %>% filter(new_col == TRUE & region == 'GB'))$year)) %>% setNames(c('year', 'GB'))

In [ ]:
it <- as.data.frame(table((merged %>% mutate(new_col = lapply(genres, function (x) 'War' %in% x)) %>% filter(new_col == TRUE & region == 'IT'))$year)) %>% setNames(c('year', 'IT'))

In [ ]:
ww2_countries <- merge(merge(merge(de, us, by='year'), gb, by='year'), it, by = 'year')

In [ ]:
ww2_countries

In [ ]:
options(repr.plot.width=8, repr.plot.height=4, repr.plot.res = 300)

In [ ]:
ggplot(ww2_countries, aes(x=year)) + geom_line(aes(y = DE, colour = "Germany"), group = 1) +
geom_line(aes(y = GB, colour = "Great Britain"), group = 1) + 
geom_line(aes(y = US, colour = "USA"), group = 1) +
geom_line(aes(y = IT, colour = "Italy"), group = 1) +
scale_color_manual(name = "Countries", values = c("Germany" = "firebrick4", "Great Britain" = "dodgerblue4", "USA" = "firebrick1", "Italy" = "forestgreen")) +
ylab("War movies screened") +
xlab("")

In [ ]:
head(merged, 10)

In [ ]:
gc()

# Title crew table

In [ ]:
head(data_crew, -20)

Replace with NAs

In [ ]:
data_crew <- data_crew %>% mutate(across(where(is.character), ~na_if(.,"\\N")))

Split the name lists

In [ ]:
data_crew <- data_crew %>% mutate(directors = strsplit(directors, split=','), writers = strsplit(writers, split=','))